In [4]:
### THE PDB FILE MUST HAVE "TER" IN BETWEEN PEPTIDE CHAINS TO AVOID MDTRAJ CREATING ANGLES BETWEEN THE ATOMS OF THE FINAL PEPTIDE ON ONE CHAIN.. AND THE N/C OF A PEPTIDE ON THE NEXT CHAIN ###

#Import library
import mdtraj as md

#Define function
def adjust_atoms_list(arr):
    ''' Takes in an array of atom numbers with 0 indexing (which MDTraj does) and returns a string 
    of comma separated atom numbers with 1 indexing (which PDB / PLUMED does)'''
    
    adj_arr = list(map(int, map(lambda x: x+1, arr)))
    adj_atoms_list = ','.join([str(s) for s in adj_arr])
    return adj_atoms_list

#Load trajectory
traj = md.load("4_peptides_v6_TER.pdb")
top = traj.topology

#Define alpha carbons (with 1-indexing) and psi/phi angle arrays
alpha_carbons = [atom.index +1 for atom in top.atoms if atom.name == 'CA']
psi_angles = md.compute_psi(traj)[0]
phi_angles = md.compute_phi(traj)[0]

#Create plumed input file
with open("plumed.dat", "w") as f:
    
    #Add psi and phi angle collective variables
    for i in range(len(psi_angles)):
        psi_atoms_list = adjust_atoms_list(psi_angles[i])
        phi_atoms_list = adjust_atoms_list(phi_angles[i])
        f.write(f"phi_{i+1}: TORSION ATOMS={phi_atoms_list}\n")
        f.write(f"psi_{i+1}: TORSION ATOMS={psi_atoms_list}\n")

    #Add distance collective variables
    counter=0
    for i in range(len(alpha_carbons)):
        for j in range(i+1, len(alpha_carbons)):
            f.write(f"d{counter+1}: DISTANCE ATOMS={alpha_carbons[i]},{alpha_carbons[j]}\n")
            counter += 1

    #Add final print line
    f.write(f"\nPRINT ARG=" + ','.join([f"phi_{i+1},psi_{i+1}" for i in range(len(psi_angles))]) + ',' + ','.join([f"d{k+1}" for k in range(counter)]) + " FILE=COLVAR STRIDE=1")


In [7]:

%%bash
plumed driver --plumed plumed.dat --ixyz xtb_test_5ns.trj --box 120,120,120
echo "------------------------------"
echo "PLUMED SUCCESSFUL"
echo "------------------------------"



PLUMED: PLUMED is starting
PLUMED: Version: 2.9.2 (git: Unknown) compiled on Sep  4 2024 at 12:23:39
PLUMED: Please cite these papers when using PLUMED [1][2]
PLUMED: For further information see the PLUMED web page at http://www.plumed.org
PLUMED: Root: /home/mjg120/miniconda3/envs/ase-v1/lib/plumed
PLUMED: For installed feature, see /home/mjg120/miniconda3/envs/ase-v1/lib/plumed/src/config/config.txt
PLUMED: Molecular dynamics engine: driver
PLUMED: Precision of reals: 8
PLUMED: Running over 1 node
PLUMED: Number of threads: 1
PLUMED: Cache line size: 512
PLUMED: Number of atoms: 492
PLUMED: File suffix: 
PLUMED: FILE: plumed.dat
PLUMED: Action TORSION
PLUMED:   with label phi_1
PLUMED:   between atoms 6 27 28 30
PLUMED:   using periodic boundary conditions
PLUMED: Action TORSION
PLUMED:   with label psi_1
PLUMED:   between atoms 1 4 6 27
PLUMED:   using periodic boundary conditions
PLUMED: Action TORSION
PLUMED:   with label phi_2
PLUMED:   between atoms 30 51 52 54
PLUMED:   using p